In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F

# Load Phi-2 model and tokenizer
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define intents and examples
intents = {
    "Buy": "I want to purchase the premium plan.",
    "Cancel": "I’d like to cancel my subscription.",
    "Complaint": "The service isn’t working as expected.",
    "Billing Inquiry": "Can you explain the charges on my bill?",
    "General Information": "What are your business hours?"
}

# Function to classify intent with confidence scores
def classify_intent(user_input):
    # Define cancellation-specific keywords
    cancellation_keywords = ["cancel", "unsubscribe", "stop my service", "end my subscription"]

    # Check for explicit cancellation-related keywords
    if any(keyword in user_input.lower() for keyword in cancellation_keywords):
        return "Cancellation", 1.0  # High confidence for explicit matches

    # Prepare the prompt
    prompt = (
        "Instruct: Based on the following customer query, classify the intent into one of "
        f"these categories: {', '.join(intents.keys())}.\n"
        f"Query: {user_input}\nOutput:"
    )

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate logits for the last token
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])

    # Extract logits for the last generated token
    logits = outputs.logits[:, -1, :]

    # Compare logits with intent tokens
    intent_logits = {intent: tokenizer.encode(intent, add_special_tokens=False)[0] for intent in intents.keys()}
    scores = {intent: logits[0, token_id].item() for intent, token_id in intent_logits.items()}

    # Select the intent with the highest score
    predicted_intent = max(scores, key=scores.get)
    confidence_score = F.softmax(torch.tensor(list(scores.values())), dim=-1).max().item()

    return predicted_intent, confidence_score




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import time

# Interactive System
if __name__ == "__main__":
    print("Customer-Agent Intent Classification System with Confidence Scores")
    print("Defined Intents:", ", ".join(intents.keys()), "\n")

    while True:
        user_input = input("Enter a customer query (or type 'exit' to quit): ")
        if user_input.lower() == "exit":
            print("Exiting system. Goodbye!")
            break

        # Measure start time
        start_time = time.time()

        # Get intent and confidence
        intent, confidence = classify_intent(user_input)

        # Measure end time and calculate latency
        end_time = time.time()
        latency = end_time - start_time

        print(f"Predicted Intent: {intent}")
        print(f"Confidence Score: {confidence:.2f}")
        print(f"Response Latency: {latency:.3f} seconds\n")


Customer-Agent Intent Classification System with Confidence Scores
Defined Intents: Buy, Cancel, Complaint, Billing Inquiry, General Information 

Enter a customer query (or type 'exit' to quit): I want to cancel my service
Predicted Intent: Cancellation
Confidence Score: 1.00
Response Latency: 0.000 seconds

Enter a customer query (or type 'exit' to quit): i want to get a new phone
Predicted Intent: Buy
Confidence Score: 0.98
Response Latency: 6.452 seconds

Enter a customer query (or type 'exit' to quit): i want to purchase a new laptop for my son
Predicted Intent: Buy
Confidence Score: 1.00
Response Latency: 5.030 seconds

Enter a customer query (or type 'exit' to quit): I am annoyed by poor customer service
Predicted Intent: Complaint
Confidence Score: 0.81
Response Latency: 4.846 seconds

Enter a customer query (or type 'exit' to quit): i am frustrated by slow internet
Predicted Intent: Complaint
Confidence Score: 0.82
Response Latency: 5.922 seconds

Enter a customer query (or ty